In [6]:
from model.model import create_model_fn
from data.triplet_generator import TripletGenerator
import tensorflow as tf
import numpy as np

from keras.models import load_model

NB_CLASSES_META_LEARNING = 10

In [2]:
_file_path='/data/whales/train.csv'
_image_train_path='/data/whales/train'
_image_test_path='/data/whales/test'
#
_file_path='/Users/deniz/Documents/whales/train.csv'
_image_train_path='/Users/deniz/Documents/whales/train'
_image_test_path='/Users/deniz/Documents/whales/test'


In [3]:
model_params = tf.contrib.training.HParams(
    fit=True,
    file_path=_file_path,
    image_train_path=_image_train_path,
    image_test_path=_image_test_path,
    model_architecture='convnet',
    tmp_data_path="/tmp/whales/",
    batch_size=None,
    nb_epochs=None,
    lr_rate=0.02,
    lr_policy=None,
    pretrained=True,
    nb_layers_to_freeze=0,
    loss=None,
    embedding_hidden_dim=64,
    triplet_margin=None,
    nb_classes=10,
    dropout=0.3)
model_params.add_hparam('image_dim', (224, 224, 3))
model_params

HParams([('batch_size', None), ('dropout', 0.3), ('embedding_hidden_dim', 64), ('file_path', '/Users/deniz/Documents/whales/train.csv'), ('fit', True), ('image_dim', (224, 224, 3)), ('image_test_path', '/Users/deniz/Documents/whales/test'), ('image_train_path', '/Users/deniz/Documents/whales/train'), ('loss', None), ('lr_policy', None), ('lr_rate', 0.02), ('model_architecture', 'convnet'), ('nb_classes', 10), ('nb_epochs', None), ('nb_layers_to_freeze', 0), ('pretrained', True), ('tmp_data_path', '/tmp/whales/'), ('triplet_margin', None)])

In [4]:
model = create_model_fn(model_params)

INFO:model.model:pretrained..


Tensor("input_1:0", shape=(?, 224, 224, 3), dtype=float32)
Tensor("conv0_relu/Relu6:0", shape=(?, 111, 111, 64), dtype=float32)
Tensor("conv1_relu/Relu6:0", shape=(?, 55, 55, 64), dtype=float32)


INFO:model.model:append the top to the structure..


Tensor("conv2_relu/Relu6:0", shape=(?, 27, 27, 64), dtype=float32)
Tensor("conv3_relu/Relu6:0", shape=(?, 13, 13, 64), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 111, 111, 64)      1728      
_________________________________________________________________
conv0_bn (BatchNormalization (None, 111, 111, 64)      256       
_________________________________________________________________
conv0_relu (ReLU)            (None, 111, 111, 64)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 55, 55, 64)        36864     
_________________________________________________________________
conv1_bn (BatchNormalization (None, 55, 55, 64)        256       
________

INFO:tensorflow:None


In [8]:
data_params = {
    'featurewise_center': False,
    'featurewise_std_normalization': False,
    'rotation_range': 20,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'zoom_range': 0.2,
    'shear_range': 0.4,
   }
NB_CLASSES_META_LEARNING = 10

In [9]:
train_generator = TripletGenerator(file_path=model_params.file_path,
                                   image_path=model_params.image_train_path,
                                   image_test_path=model_params.image_test_path,
                                   nb_classes_batch=NB_CLASSES_META_LEARNING,
                                   nb_images_per_class_batch=5,
                                   validation_split=0.2,
                                   subset='training',
                                   **data_params)
eval_generator = TripletGenerator(file_path=model_params.file_path,
                                  image_path=model_params.image_train_path,
                                  image_test_path=model_params.image_test_path,
                                  nb_classes_batch=NB_CLASSES_META_LEARNING,
                                  nb_images_per_class_batch=5,
                                  validation_split=0.2,
                                  subset='eval',
                                  **data_params)

INFO:data.triplet_generator:exclude_class: ['new_whale']
INFO:data.triplet_generator:class_weight_type: None
INFO:data.triplet_generator:9664 instances excluded
INFO:data.triplet_generator:data has shape: (12415, 2)
INFO:data.triplet_generator:{'featurewise_center': False, 'featurewise_std_normalization': False, 'rotation_range': 20, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'zoom_range': 0.2, 'shear_range': 0.4, 'preprocessing_function': <function _preprocess_input at 0x12d825488>}
INFO:data.triplet_generator:exclude_class: ['new_whale']
INFO:data.triplet_generator:class_weight_type: None
INFO:data.triplet_generator:9664 instances excluded
INFO:data.triplet_generator:data has shape: (3282, 2)
INFO:data.triplet_generator:{'featurewise_center': False, 'featurewise_std_normalization': False, 'rotation_range': 20, 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'zoom_range': 0.2, 'shear_range': 0.4, 'preprocessing_function': <function _preprocess_input at 0x12d825488>}


In [10]:
len(train_generator), len(eval_generator)

(400, 100)

In [11]:
def train(model_params, train_generator, eval_generator, outer_step_size = 0.02, steps=5):
    def fn(_gen):
        for a, b in _gen:
            yield a, b
    nb_steps_per_epoch = len(train_generator)
    nb_steps_per_epoch_val = len(eval_generator)
    _generator = fn(train_generator)
    _eval_generator = fn(eval_generator)
    NB_EPOCHS = 10
    # reptile training loop
    for i in range(NB_EPOCHS):
        for _ in range(nb_steps_per_epoch):
            
            weights_before = model.weights
            
            # sample a new task. need to take k steps.
            x, y = next(_generator)
            # randomize bc generator does not
            ix = list(range(x.shape[0]))
            np.random.shuffle(ix)
            x, y = x[ix], y[ix]

            batch_size = x.shape[0] // steps
            # take k steps
            for s in range(steps):   
                _x, _y = x[s * batch_size: (s+1) * batch_size,:,:,:], y[s * batch_size: (s+1) * batch_size,:]
                model.train_on_batch(_x, _y)
            weights_after = model.weights
            
            # meta update after task training.
            for w_ix in range(len(weights_before)):
                model.weights[i] = (weights_before[i] +
                                    (weights_after[i] - weights_before[i]) * outer_step_size)
            
        #eval
        print('eval')
        outs_per_batch = []
        for s in range(nb_steps_per_epoch_val):
            x, y = next(_eval_generator)
            outs = model.test_on_batch(x, y)
            outs_per_batch.append(outs)
        averages = []
        # outs: (val_loss, accuracy, top_5_accuracy)
        stateful_metric_indices = [1,2]
        for i in range(len(outs)):
            if i not in stateful_metric_indices:
                averages.append(np.average([out[i] for out in outs_per_batch]))
            else:
                # error on the last batch.
                averages.append(np.float64(outs_per_batch[-1][i]))
        print("epoch: {}, metrics: {}".format(i, averages))
        
    #TODO: eval metric. early stopping based on that.
    model.save(model_params.tmp_data_path + 'meta_learning_model.h5')
    return model, averages

In [13]:
model, all_outs = train(model_params, train_generator, eval_generator)

KeyboardInterrupt: 

In [ ]:
def generate_nn_preds():
    x, img_names = generator.get_test_images_and_names()

In [ ]:
# test time.
def test(model_params, nn_classifier_predictions, nb_classes_meta=NB_CLASSES_META_LEARNING):
    """
    nn_classifier_predictions: [test_data_len, nb_classes]
    """
    # test images and names
    gen = Generator(file_path=model_params.file_path,
                    image_path=model_params.image_train_path,
                    image_test_path=model_params.image_test_path,
                    batch_size=model_params.batch_size)
    x, img_names = generator.get_test_images_and_names()
    nn_classifier_predictions_class_ix = np.argsort(-nn_classifier_predictions)[:nb_classes_meta]
    assert len(x), len(nn_classifier_predictions_class_ix)
    # load the meta-training model
    model = load_model(model_params.tmp_data_path + 'meta_learning_model.h5')
    
    # each prediction becomes a task downstream
    ensemble_preds = []
    for i,task_class_ids in enumerate(nn_classifier_predictions_class_ix):
        weights_before = model.weights
        # training - training images
        batch_x, batch_y = test_generator.get_train_image_from_class_ix(task_class_ids)
        model.train_on_batch(batch_x, batch_y)
        # predict - test image
        outs = model.predict_on_batch(x[i])
        # take top 5 predictions
        pred_class_ix = np.argmax(-outs)[:5]
        ensemble_preds.append(pred_class_ix)
        # revert back for the next task
        model.weights = weights_before
    return np.vstack(ensemble_preds)    

In [15]:
arr= np.random.random([10,2])
for a in arr:
    print(a)

[0.30285652 0.80148075]
[0.07503177 0.27112158]
[0.14037516 0.54531099]
[0.10841571 0.93512427]
[0.11505544 0.24019544]
[0.76894898 0.39560027]
[0.86139979 0.17824399]
[0.81073242 0.93174129]
[0.82807975 0.89988127]
[0.79756544 0.58660252]


In [7]:
test_generator = TripletGenerator(file_path=model_params.file_path,
                                   image_path=model_params.image_train_path,
                                   image_test_path=model_params.image_test_path,
                                   nb_classes_batch=NB_CLASSES_META_LEARNING,
                                   shuffle=False,
                                   nb_images_per_class_batch=1, # test is one-shot learning
)

INFO:data.triplet_generator:exclude_class: ['new_whale']
INFO:data.triplet_generator:class_weight_type: None
INFO:data.triplet_generator:9664 instances excluded
INFO:data.triplet_generator:data has shape: (15697, 2)
INFO:data.triplet_generator:{'preprocessing_function': <function _preprocess_input at 0x12f200400>}


In [8]:
dt = test_generator.get_image_from_class_ix([0,1,2,3,4])

In [11]:
dt[1]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)